In [195]:
# Basic SciPy packages
import numpy as np
import pandas as pd

# Stats Packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Graphing packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
#%matplotlib inline
%matplotlib tk

#Load my random dataset
import CreateDataSet


In [196]:
#Generate Dataset

cvd_dataset = CreateDataSet.make_cvd_dataset()
cvd_dataset

,Power,GrowthRate
0,8,5004.586161
1,8,5000.490097
2,8,5025.169991
3,8,5035.613233
4,10,5072.326279
5,10,5060.799200
6,10,5058.756898
7,10,5060.765850
8,12,5106.982790
9,12,5109.928801


In [197]:
## Save dataset to csv to lock it in. 
#cvd_dataset.to_csv("CVD_Dataset_EDA_ANOVA.csv")
cvd_dataset = pd.read_csv("CVD_Dataset_EDA_ANOVA.csv")

In [198]:
sns.boxplot(x = "Power", y = "GrowthRate", data = cvd_dataset)

In [199]:
## Create my own ANOVA
cvd_dataset_initial = cvd_dataset

In [200]:
runID = pd.Series(data=[1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4],name='RunID')
cvd_dataset['RunID'] = runID
cvd_dataset

,Power,GrowthRate,RunID
0,8,5000.000098,1
1,8,5012.664928,2
2,8,5018.486875,3
3,8,5000.003111,4
4,10,5050.553024,1
5,10,5051.092497,2
6,10,5051.348757,3
7,10,5080.248275,4
8,12,5106.268943,1
9,12,5101.476174,2


In [201]:
cvd_pivot = cvd_dataset.pivot(index = 'RunID', columns = 'Power', values = 'GrowthRate')
cvd_pivot

Power,8,10,12,14
RunID,,,,
1,5000.000098,5050.553024,5106.268943,5150.872442
2,5012.664928,5051.092497,5101.476174,5152.530164
3,5018.486875,5051.348757,5106.513395,5159.038556
4,5000.003111,5080.248275,5107.977620,5150.521528


In [202]:
## Between samples is across the row
## Within samples is down the column


# Define number of replicates within treatment:
n = cvd_pivot.count(axis=1)
# Determine number of treatments:
a = cvd_pivot.count()


n = n.iloc[0]
a = a.iloc[0]

# Total number of datapoints
TotalN = n*a
TotalN


#dft is degrees of freedom between sample set
#dfe is degrees of freedom within samples
#dftotal is degrees of freedom total 

dft = a - 1
dfe = TotalN - a
dftotal = TotalN-1


In [203]:
#Compute sum under i-th treatment
yi_sum = cvd_pivot.sum()
yi_sum

Power
8     20031.155012
10    20233.242553
12    20422.236133
14    20612.962690
dtype: float64

In [204]:
#Compute average under i-th treatment 
yi_avg = cvd_pivot.mean()
yi_avg

Power
8     5007.788753
10    5058.310638
12    5105.559033
14    5153.240673
dtype: float64

In [205]:
#Compute grand sum
y_sum = yi_sum.sum()
y_sum

81299.59638766023

In [206]:
# Compute grand average
y_avg = yi_avg.mean()
y_avg

5081.2247742287645

In [207]:
# Varability between treatments
SSTR = (yi_avg - y_avg)**2
SSTR = SSTR.sum()*n
SSTR

46785.410453411532

In [208]:
# Varability within treatments

SSE = cvd_pivot.sub(yi_avg.values) ** 2
SSE = SSE.sum().sum()
SSE

972.5727188495458

In [209]:
## Total corrected sum of squares

SST = SSE - SSTR
SST

-45812.837734561988

In [210]:
## Mean Square Error

MSE = SSE/(dfe)
MSE

81.04772657079549

In [211]:
## Mean Square Treatments
MSTR = SSTR/(dft)
MSTR

15595.136817803845

In [212]:
#F ratio 
F = MSTR/MSE
F

192.41917667093887

In [213]:
p = stats.f.sf(F, dfe, dft)
p

0.00054615547385569035

In [214]:
eta_sqr = SSTR/SST
eta_sqr

-1.0212292616424374

In [215]:
ome_sqr = (SSTR - (a*MSE))/ (SST+ n)
ome_sqr

-1.0142413963075547

In [219]:
print("Sum of Squares within = {} and Sum of Squares between = {}.".format(SSE, SSTR))

Sum of Squares within = 972.5727188495458 and Sum of Squares between = 46785.41045341153.


In [220]:
print("F ratio is {} and the p is {}." .format(F,p))

F ratio is 192.41917667093887 and the p is 0.0005461554738556904.


In [224]:
#Check versus built in
f_val, p_val = stats.f_oneway(cvd_pivot[8], cvd_pivot[10], cvd_pivot[12], cvd_pivot[14])  
print(f_val, p_val)

192.419176671 2.07342242061e-10


In [225]:
## Point estimator of mu_i is mu_hat plus ti hat which is yi_bar
## Mu hat is also y hat
mu_hat = yi_avg
mu_hat

Power
8     5007.788753
10    5058.310638
12    5105.559033
14    5153.240673
dtype: float64

In [226]:
##Determine treatment effects
t_hat = mu_hat - y_avg

In [108]:
cvd_pivot_residuals = cvd_pivot.sub(mu_hat, axis = 1)

In [228]:
cvd_pivot_residuals

Power,8,10,12,14
RunID,,,,
1,-7.788655,-7.757614,0.709910,-2.368231
2,4.876175,-7.218142,-4.082859,-0.710508
3,10.698122,-6.961881,0.954362,5.797883
4,-7.785642,21.937637,2.418587,-2.719144


In [230]:
# Melt back to plot
cvd_pivot_residuals_melted = cvd_pivot_residuals.melt()
cvd_pivot_residuals_melted

,Power,value
0,8,-7.788655
1,8,4.876175
2,8,10.698122
3,8,-7.785642
4,10,-7.757614
5,10,-7.218142
6,10,-6.961881
7,10,21.937637
8,12,0.709910
9,12,-4.082859


In [231]:
stats.probplot(cvd_pivot_residuals_melted['value'], plot=plt)

#-> Looks good

((array([-1.72352605, -1.26569652, -0.97848645, -0.75533862, -0.56472935,
         -0.39279634, -0.23181469, -0.07666006,  0.07666006,  0.23181469,
          0.39279634,  0.56472935,  0.75533862,  0.97848645,  1.26569652,
          1.72352605]),
  array([ -7.78865538,  -7.78564186,  -7.75761431,  -7.21814166,
          -6.96188073,  -4.08285905,  -2.7191442 ,  -2.36823065,
          -0.71050816,   0.70991016,   0.95436224,   2.41858665,
           4.87617512,   5.797883  ,  10.69812212,  21.93763671])),
 (7.931622867907242, -1.1434727799641058e-13, 0.92636395404928951))

In [232]:
cvd_predict_residual = cvd_pivot_residuals_melted

In [233]:
for power_setting, power_avg in mu_hat.iteritems():
    cvd_predict_residual.loc[cvd_predict_residual['Power'] == power_setting, 'Power'] =\
    power_avg

In [234]:
plt.scatter(cvd_predict_residual['Power'], cvd_predict_residual['value'])
plt.xlabel('Predicted Growth Rate')
plt.ylabel('Residuals')
plt.show()